
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/02_Broadcasting_Magic.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🥋 Lekcja 2: Broadcasting Magic (Matematyka bez pętli)

Jak dodać wektor `[1, 2, 3]` do każdego wiersza macierzy `[[0, 0, 0], [10, 10, 10]]`?
W C++ pisałbyś pętlę. W PyTorch dzieje się to "samo".

**Zasady Broadcastingu:**
PyTorch porównuje wymiary dwóch tensorów **od prawej do lewej**:
1.  Jeśli wymiary są równe -> OK.
2.  Jeśli jeden z wymiarów ma rozmiar **1** -> Rozciągnij go (wirtualnie) do rozmiaru drugiego.
3.  Jeśli jednego wymiaru brakuje -> Dopisz **1** z lewej strony.

Jeśli żadna zasada nie pasuje -> Błąd.

**Cel lekcji:**
Zrozumieć, jak PyTorch "oszukuje" pamięć, udając, że małe tensory są duże, żeby wykonać obliczenia błyskawicznie.

In [1]:
import torch

# 1. PRZYKŁAD PODSTAWOWY
# Macierz (2 wiersze, 3 kolumny)
A = torch.tensor([[0, 0, 0], 
                  [10, 10, 10]])

# Wektor (3 elementy)
B = torch.tensor([1, 2, 3])

print(f"Kształt A: {A.shape}")
print(f"Kształt B: {B.shape}")

# Dodawanie
C = A + B

print("\n--- WYNIK A + B ---")
print(C)
print("Co się stało? Wektor B został 'dodany' do każdego wiersza A osobno.")

Kształt A: torch.Size([2, 3])
Kształt B: torch.Size([3])

--- WYNIK A + B ---
tensor([[ 1,  2,  3],
        [11, 12, 13]])
Co się stało? Wektor B został 'dodany' do każdego wiersza A osobno.


## Co się stało pod maską? (Wizualizacja Zasad)

Analiza kształtów od prawej:
*   A: `(2, 3)`
*   B: `(   3)`

1.  Wyrównanie do prawej: Ostatni wymiar to `3` vs `3`. Pasuje.
2.  Brakujący wymiar: B nie ma pierwszego wymiaru. PyTorch traktuje go jako `(1, 3)`.
3.  Rozciąganie jedynek: Wymiar `1` w B jest rozciągany do `2` (żeby pasował do A).

Efektywnie B staje się:
`[[1, 2, 3], [1, 2, 3]]` (ale tylko wirtualnie, w pamięci to nadal 3 liczby!).

In [2]:
# 2. PRZYKŁAD TRUDNY (Kolumna + Wiersz)
# Wektor kolumnowy (3, 1)
col = torch.tensor([[1], 
                    [2], 
                    [3]])

# Wektor wierszowy (1, 4) (lub po prostu 4)
row = torch.tensor([10, 20, 30, 40])

print(f"Col: {col.shape}")
print(f"Row: {row.shape}")

# Wynik? Macierz 3x4!
# Col rozciąga się w prawo. Row rozciąga się w dół.
grid = col + row

print("\n--- WYNIK (Siatka) ---")
print(grid)
print(f"Nowy kształt: {grid.shape}")

Col: torch.Size([3, 1])
Row: torch.Size([4])

--- WYNIK (Siatka) ---
tensor([[11, 21, 31, 41],
        [12, 22, 32, 42],
        [13, 23, 33, 43]])
Nowy kształt: torch.Size([3, 4])


## .expand() vs .repeat() (Pamięć)

To jest test na Seniora.
Chcesz powielić tensor. Czego użyjesz?

*   **`.repeat(2, 2)`**: Fizycznie kopiuje dane. Zajmuje nową pamięć.
*   **`.expand(2, 2)`**: Tworzy **Widok (View)**. Ustawia `stride=0` na rozciąganym wymiarze. **Zużycie pamięci = 0.**

Zawsze używaj `expand()`, jeśli tylko potrzebujesz odczytać dane (do matematyki).

In [3]:
t = torch.tensor([[1], [2], [3]]) # (3, 1)

print("--- ORYGINAŁ ---")
print(f"Shape: {t.shape}, Stride: {t.stride()}")
print(f"Adres pamięci: {t.untyped_storage().data_ptr()}")

# Używamy EXPAND (powiększamy do 3x4)
t_expanded = t.expand(3, 4)

print("\n--- EXPAND (Wirtualna kopia) ---")
print(f"Shape: {t_expanded.shape}")
print(f"Stride: {t_expanded.stride()}") 
# Zauważ stride=(1, 0). 0 oznacza: "żeby przejść do następnej kolumny, przesuń się o 0 bajtów".
# Czyli czytamy ciągle tę samą liczbę!

print(f"Adres pamięci: {t_expanded.untyped_storage().data_ptr()}")
if t.untyped_storage().data_ptr() == t_expanded.untyped_storage().data_ptr():
    print("✅ To ten sam obszar pamięci! Zero kosztów.")

# Używamy REPEAT
t_repeated = t.repeat(1, 4)
print("\n--- REPEAT (Fizyczna kopia) ---")
print(f"Adres pamięci: {t_repeated.untyped_storage().data_ptr()}")
print("❌ Adres jest inny. Zaalokowano nową pamięć.")

--- ORYGINAŁ ---
Shape: torch.Size([3, 1]), Stride: (1, 1)
Adres pamięci: 6545664508352

--- EXPAND (Wirtualna kopia) ---
Shape: torch.Size([3, 4])
Stride: (1, 0)
Adres pamięci: 6545664508352
✅ To ten sam obszar pamięci! Zero kosztów.

--- REPEAT (Fizyczna kopia) ---
Adres pamięci: 6545664508416
❌ Adres jest inny. Zaalokowano nową pamięć.


## Praktyczne Zastosowanie: Macierz Odległości (Pairwise Distance)

Masz 3 punkty A i 2 punkty B. Chcesz policzyć odległość każdego A do każdego B.
Bez pętli `for`.

*   A: `(3, 2)` -> Zmieniamy na `(3, 1, 2)`
*   B: `(2, 2)` -> Zmieniamy na `(1, 2, 2)`
*   Różnica: `(3, 2, 2)` -> Każdy z każdym!

In [4]:
# Punkty A (np. Klienci)
A = torch.tensor([[0., 0.], 
                  [1., 1.], 
                  [2., 2.]]) # (3, 2)

# Punkty B (np. Sklepy)
B = torch.tensor([[0., 1.], 
                  [10., 10.]]) # (2, 2)

# Chcemy wynik (3, 2) - odległości

# 1. Unsqueeze (Dodajemy wymiary "dummy" dla broadcastingu)
# A: (3, 1, 2)
# B: (1, 2, 2)
diff = A.unsqueeze(1) - B.unsqueeze(0)

print(f"Kształt różnicy: {diff.shape}")
# (3, 2, 2) -> (3 klientów, 2 sklepy, 2 współrzędne x/y)

# 2. Kwadrat i Suma (Norma L2)
dist_sq = diff ** 2
dist_sum = dist_sq.sum(dim=2) # Sumujemy po współrzędnych (x+y)
dist = torch.sqrt(dist_sum)

print("\n--- MACIERZ ODLEGŁOŚCI ---")
print(dist)
# Wiersz 0: Odległość Klienta 0 do Sklepu 0 i Sklepu 1
# Wiersz 1: Odległość Klienta 1 ...

Kształt różnicy: torch.Size([3, 2, 2])

--- MACIERZ ODLEGŁOŚCI ---
tensor([[ 1.0000, 14.1421],
        [ 1.0000, 12.7279],
        [ 2.2361, 11.3137]])


## 🥋 Black Belt Summary

1.  **Zasada prawej ręki:** Wymiary są dopasowywane od prawej strony.
2.  **Jedyneczka to joker:** Wymiar o rozmiarze `1` dopasowuje się do wszystkiego.
3.  **Expand > Repeat:** `expand` manipuluje tylko *stride* (krokiem), `repeat` kopiuje dane.
4.  **Unsqueeze jest przyjacielem:** Często dodajemy `None` lub `.unsqueeze()`, żeby "przygotować" tensor pod broadcasting (np. zmieniając `(N)` na `(N, 1)`).